In [2]:
import pandas as pd 
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import BinaryAccuracy

In [3]:
label = pd.read_csv('devset_images_gt.csv') 
# Preview the first 5 lines of the loaded data 
label_array = label.values


In [4]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Đường dẫn tới folder chứa ảnh
folder_path = "C:/Users/minhd/FPTU lab/DPL302m/Kaggle/2024-sum-dpl-302-m/devset_images/devset_images"

# Hàm để đọc và xử lý từng ảnh
def load_and_preprocess_image(image_path, target_size=(256, 256)):
    image = Image.open(image_path)
    image = image.resize(target_size)
    image = np.array(image)
    if image.shape[-1] == 4:  # Nếu ảnh có kênh alpha (RGBA), chuyển về RGB
        image = image[:, :, :3]
    image = image / 255.0  # Chuẩn hóa ảnh về khoảng [0, 1]
    return image

# Đọc tất cả các ảnh trong folder
images = []
Y_train = []

# List of allowed file extensions
allowed_extensions = ['.jpg', '.png', '.gif']

# Giả sử label_array là một mảng các tên file và nhãn tương ứng
# Cập nhật lại danh sách các file ảnh với các định dạng khác nhau
for filename in os.listdir(folder_path):
    # Check if the file has one of the allowed extensions
    if os.path.splitext(filename)[1].lower() in allowed_extensions:
        # Extract the base name without the extension to match with label_array
        basename = os.path.splitext(filename)[0]
        # Find the corresponding label for the current image file
        for i in label_array:
            if basename == str(i[0]):
                image_path = os.path.join(folder_path, filename)
                image = load_and_preprocess_image(image_path)
                images.append(image)
                Y_train.append(i[1])
                break  # Break the inner loop once the label is found

In [5]:
images = np.array(images)
Y_train = np.array(Y_train)

In [6]:
train_images, val_images, train_labels, val_labels = train_test_split(images, Y_train, test_size=0.2, random_state=42)

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.applications import ResNet50V2


In [34]:
model = Sequential()

In [36]:
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [37]:
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
base_model.trainable = False

In [40]:

model.compile('adam', loss= 'categorical_crossentropy', metrics=['accuracy'])

In [42]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ ?                         │ 23,564,800 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ global_average_pooling2d        │ ?                         │          0 │
│ (GlobalAveragePooling2D)        │                           │  (unbuilt) │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten_1 (Flatten)             │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_2 (Dense)                 │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_1 (Dropout)             │ ?                         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_3 (Dense)                 │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 23,564,800 (89.89 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,564,800 (89.89 MB)

In [44]:
history = model.fit(
    train_images, train_labels,
    epochs= 10,
    batch_size=32,
    validation_data=(val_images, val_labels)
)

Epoch 1/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - accuracy: 0.7991 - loss: 35.5936 - val_accuracy: 0.8248 - val_loss: 36.9680
Epoch 2/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.8722 - loss: 33.7516 - val_accuracy: 0.8788 - val_loss: 35.9026
Epoch 3/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.8911 - loss: 33.3482 - val_accuracy: 0.8807 - val_loss: 35.8752
Epoch 4/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 214s 2s/step - accuracy: 0.9028 - loss: 32.9243 - val_accuracy: 0.8977 - val_loss: 36.7292
Epoch 5/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.9130 - loss: 32.2934 - val_accuracy: 0.8977 - val_loss: 35.8277
Epoch 6/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.9192 - loss: 31.2115 - val_accuracy: 0.9062 - val_loss: 36.5967
Epoch 7/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - accuracy: 0.9355 - loss: 29.8159 - val_accuracy: 0.8977 - val_loss: 36.2699
Epoch 8/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.9359 - loss: 30.

In [46]:
loss, accuracy = model.evaluate(val_images, val_labels)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')

33/33 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.8857 - loss: 35.1029
Validation loss: 36.44007110595703
Validation accuracy: 0.8929924368858337


In [ ]:
# Đường dẫn tới thư mục chứa ảnh
folder_path = "C:/Users/minhd/FPTU lab/DPL302m/Kaggle/2024-sum-dpl-302-m/testset_images/testset_images"

# Đọc và xử lý ảnh, sau đó dự đoán với mô hình
def predict_image(model, img_path, target_size=(256, 256)):
    image = Image.open(img_path)
    image = image.resize(target_size)
    image = np.array(image)
    if image.shape[-1] == 4:  # Nếu ảnh có kênh alpha (RGBA), chuyển về RGB
        image = image[:, :, :3]
    image = image / 255.0
    image = np.expand_dims(image, axis=0)  # Thêm chiều batch vào hình ảnh
    prediction = model.predict(image)
    return 1 if prediction[0][0] >= 0.5 else 0

test_label = []
id_test = []

# Danh sách các phần mở rộng file hợp lệ
allowed_extensions = ['.jpg', '.png', '.gif']

# Giả sử label_array là một mảng các tên file và nhãn tương ứng
# Cập nhật lại danh sách các file ảnh với các định dạng khác nhau
for filename in os.listdir(folder_path):
    # Kiểm tra nếu file có phần mở rộng hợp lệ
    if os.path.splitext(filename)[1].lower() in allowed_extensions:
        # Lấy tên file không có phần mở rộng để khớp với label_array
        basename = os.path.splitext(filename)[0]
        id_test.append(basename)
        # Tìm nhãn tương ứng cho file ảnh hiện tại
        image_path = os.path.join(folder_path, filename)
        output = predict_image(model, image_path)
        test_label.append(output)

print(test_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━

In [ ]:
submit = pd.DataFrame({'id': id_test, 'label': test_label})
submit.head()

In [ ]:
results_csv_path = 'submitResnet50V2.csv'
submit.to_csv(results_csv_path, index=False)